# Action Provider Demo: send requests directly to an AP endpoint

### Select an Action Provider Host

In [ ]:
"""Use Diaspora AP directly through endpoint."""

from __future__ import annotations

import json

import requests
from diaspora_event_sdk import Client as GlobusClient

ap_host = 'http://127.0.0.1:8000/'
# ap_host = 'https://flask-service.ml22sevubfnks.us-east-1.cs.amazonlightsail.com/'
print('AP endpoint selected:', ap_host)

### Retrieve Credential

In [ ]:
c = GlobusClient()
print("User's OpenID:", c.subject_openid)

tokens = c.login_manager._token_storage.get_token_data(
    '2b9d2f5c-fa32-45b5-875b-b24cd343b917',
)

access_token = tokens['access_token']
print("User's access token:", access_token)

### Select a Topic

In [ ]:
topic = 'topic' + c.subject_openid[-12:]
print(c.register_topic(topic))
print(c.list_topics())
print('Topic to produce/consume:', topic)

### 2.1.1 Produce Messages to AP without msg keys

In [ ]:
headers = {
    'authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json',
}

data = {
    'request_id': '112',
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
    },
}

In [ ]:
response = requests.post(f'{ap_host}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 2.1.2: Error Case: `msgs` does not exist

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'produce',
        'topic': topic,
    },
}
response = requests.post(f'{ap_host}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)['details']['error']
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 2.1.3: Error Case: `msgs` is empty

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [],
    },
}
response = requests.post(f'{ap_host}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)['details']['error']
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 2.1.4: Error Case: the topic does not exist (takes 10 seconds)

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'produce',
        'topic': '__badtopic',
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
    },
}
response = requests.post(f'{ap_host}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)['details']['error']
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 2.2.1 Produce Messages to AP with a Single Key

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
        'keys': 'my-key',
    },
}
response = requests.post(f'{ap_host}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 2.3.1 Produce Messages to AP with a List of Keys

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
        'keys': ['my-key1', 'my-key2', 'my-key3'],
    },
}
response = requests.post(f'{ap_host}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 2.3.2 Error case: the length of `keys` does not match the length of `msgs`

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
        'keys': ['my-key1'],
    },
}
response = requests.post(f'{ap_host}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)['details']['error']
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 3.1.1: Consume Messages since the Current Timestamp

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'consume',
        'topic': topic,
    },
}
response = requests.post(f'{ap_host}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 3.1.2: Consume Messages since a Timestamp

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'consume',
        'topic': topic,
        'ts': 1715930522000,
    },
}
response = requests.post(f'{ap_host}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 3.1.3: Error case: the topic does not exist

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'consume',
        'topic': '__badtopic',
        'ts': 1715930522000,
    },
}
response = requests.post(f'{ap_host}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

In [ ]:
data = {
    'request_id': '112',
    'body': {
        'action': 'consume',
        'topic': 'dsaas',
        'ts': 1715930522000,
    },
}
response = requests.post(f'{ap_host}/run', headers=headers, json=data)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)

### 4.1.1 Check Request Status, Cancel Request, and Release Request (Not Implemented and Always Succeed)

In [ ]:
action_id = '1mX5FKSJTFMy'

response = requests.get(
    f'{ap_host}/{action_id}/status',
    headers={'authorization': f'Bearer {access_token}'},
)

# Print the response from the server
print('Response status code:', response.status_code)
response_content = response.content.decode('utf-8')
parsed_content = json.loads(response_content)
formatted_content = json.dumps(parsed_content, indent=2)
print('Response content:', formatted_content)


for endpoint in ['cancel', 'release']:
    response = requests.post(
        f'{ap_host}/{action_id}/{endpoint}',
        headers={'authorization': f'Bearer {access_token}'},
    )

    # Print the response from the server
    print('Response status code:', response.status_code)
    response_content = response.content.decode('utf-8')
    parsed_content = json.loads(response_content)
    formatted_content = json.dumps(parsed_content, indent=2)
    print('Response content:', formatted_content)